In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:
from pyspark.sql import SparkSession
import re
import xml.etree.ElementTree as ET
from pyspark.sql.functions import udf, explode, col, when, lower
from pyspark.sql.types import ArrayType, StringType
from operator import add
spark = SparkSession.builder.getOrCreate()
df = spark.read.format('xml').options(rowTag='page').load('hdfs:/wiki-whole.xml')
#df.printSchema()

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4dd39759-4669-462c-b13a-fb0bb3e9165d;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 581ms :: artifacts dl 15ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded

In [3]:
# df.printSchema()

In [4]:
import re
#write the udf and regrex function
def extract_text(text):
    try:
        match = re.findall(r'\[\[[^\[\]]+\]\]',text)
    except:
        match = []
    output = []
    for link in match:
        temp = link.split('|')[0].lower()
        if (':' in temp) and ('Category:' not in temp):
            continue
        elif '#' in temp:
            continue
        else: output.append(temp)
    return [re.sub(r'\[|\]','',a) for a in output]

In [5]:
# In[3]:

udf_extract_text = udf(lambda x:extract_text(x), ArrayType(StringType()))
#select columns that we need
df2 = df.select(col('title'), col('revision.text._VALUE').alias('links'))
#explode the links column
df2 = df2.withColumn('internal_links',explode(udf_extract_text(df2['links'])))
#lower all the rows
df2 = df2.select(lower(col('title')).alias('title'),col('internal_links').alias('internal_links'))
#drop na rows
df2 = df2.select(col('title'),col('internal_links')).na.drop()
#sort rows in ascending order
df2  = df2.select(col('title'),col('internal_links')).sort(['title','internal_links'],ascending=True)

In [6]:
# df2.printSchema()

root
 |-- title: string (nullable = true)
 |-- internal_links: string (nullable = true)



In [7]:
# df2.show(20)

23/04/29 00:26:12 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)


+--------------------+--------------------+
|               title|      internal_links|
+--------------------+--------------------+
|"hello, world!" p...|                .deb|
|"hello, world!" p...|3d computer graphics|
|"hello, world!" p...|ada (programming ...|
|"hello, world!" p...|            algol 60|
|"hello, world!" p...|application progr...|
|"hello, world!" p...|               ascii|
|"hello, world!" p...|b (programming la...|
|"hello, world!" p...|               basic|
|"hello, world!" p...|                bcpl|
|"hello, world!" p...|           bell labs|
|"hello, world!" p...|     brian kernighan|
|"hello, world!" p...|     brian kernighan|
|"hello, world!" p...|c (programming la...|
|"hello, world!" p...|c sharp (programm...|
|"hello, world!" p...|                 c++|
|"hello, world!" p...|                 c++|
|"hello, world!" p...|         catchphrase|
|"hello, world!" p...|               cobol|
|"hello, world!" p...|complex programma...|
|"hello, world!" p...|          

In [8]:
articles = df2.select(col('title')).distinct().union(df2.select(col('internal_links')).distinct())
# get all articles to generate initial ranks

In [9]:
# articles.show(20)

23/04/29 00:27:14 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
23/04/29 00:27:15 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at c

+--------------------+
|               title|
+--------------------+
|               aruba|
|                 art|
|               anime|
|antigua and barbu...|
|  economy of armenia|
|  ancient philosophy|
|       alligatoridae|
|albert of branden...|
|american airlines...|
|   american folklore|
|            april 24|
|         amblygonite|
|         brassicales|
|  batman (1989 film)|
|            chordate|
|cutaway (filmmaking)|
|           actresses|
|analysis of variance|
| ara (constellation)|
|astronomical year...|
+--------------------+
only showing top 20 rows



In [10]:
ranks = articles.rdd.map(lambda url_neighbors: (url_neighbors[0], 1.0))


23/04/29 00:28:42 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)


In [11]:
iterations = 10

In [12]:
df2_rdd = df2.rdd.groupByKey().cache()

In [13]:
# print(df2_rdd.collect()[:10])

[('cinema of germany', <pyspark.resultiterable.ResultIterable object at 0x7ff87532d580>), ('cinema of poland', <pyspark.resultiterable.ResultIterable object at 0x7ff87532d520>), ('cinema of the soviet union', <pyspark.resultiterable.ResultIterable object at 0x7ff87532d5e0>), ('ciprofloxacin', <pyspark.resultiterable.ResultIterable object at 0x7ff87532d640>), ('circle', <pyspark.resultiterable.ResultIterable object at 0x7ff87532d670>), ('circuit noise level', <pyspark.resultiterable.ResultIterable object at 0x7ff87532d6d0>), ('circuit restoration', <pyspark.resultiterable.ResultIterable object at 0x7ff87532d730>), ('circular polarization', <pyspark.resultiterable.ResultIterable object at 0x7ff87532d790>), ('circumference', <pyspark.resultiterable.ResultIterable object at 0x7ff87532d7f0>), ('circumsision', <pyspark.resultiterable.ResultIterable object at 0x7ff87532d850>)]


In [14]:
# contribution = df2_rdd.join(ranks)

In [15]:
# print(contribution.collect()[:10])

[('ezra abbot', (<pyspark.resultiterable.ResultIterable object at 0x7ff875f431c0>, 1.0)), ('ezra abbot', (<pyspark.resultiterable.ResultIterable object at 0x7ff875bcf460>, 1.0)), ('catalyst', (<pyspark.resultiterable.ResultIterable object at 0x7ff875bcf2b0>, 1.0)), ('catalyst', (<pyspark.resultiterable.ResultIterable object at 0x7ff875bcf280>, 1.0)), ('cognitive psychology', (<pyspark.resultiterable.ResultIterable object at 0x7ff875bcf1c0>, 1.0)), ('cognitive psychology', (<pyspark.resultiterable.ResultIterable object at 0x7ff875bcf1c0>, 1.0)), ('color blindness', (<pyspark.resultiterable.ResultIterable object at 0x7ff875bcfe80>, 1.0)), ('color blindness', (<pyspark.resultiterable.ResultIterable object at 0x7ff87532d190>, 1.0)), ('e. p. thompson on luddites', (<pyspark.resultiterable.ResultIterable object at 0x7ff87532d160>, 1.0)), ('list of female tennis players', (<pyspark.resultiterable.ResultIterable object at 0x7ff87532d460>, 1.0))]


In [16]:
def computeContribs(urls, rank):
    # calculate contribution for each url in urls
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)

In [17]:
from operator import add
for iteration in range(iterations):
    contribution = df2_rdd.join(ranks).flatMap(lambda x: computeContribs(
            x[1][0], x[1][1]  # type: ignore[arg-type]
        ))
    # (article0, (neighbours, rank)) -> (article, contribution)
    ranks = contribution.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)

In [ ]:
ranks_df= ranks.toDF().withColumnRenamed("_1","article").withColumnRenamed("_2","rank").sort(["rank"], ascending=False)

In [ ]:
# ranks_df.show(10)

+-----------------+------------------+
|          article|              rank|
+-----------------+------------------+
|  catholic church|22.648388248183334|
|           france|12.599248243352337|
|    new york city|10.322981159646607|
|         buddhism| 8.501049791006622|
|            islam| 6.994866345599904|
|       philosophy| 6.449267209157854|
|      linguistics| 5.304698617381028|
|     cryptography|  4.87365518769875|
|   climate change| 4.671917261754584|
|personal computer| 4.399111028276592|
+-----------------+------------------+
only showing top 10 rows



In [ ]:
ranks_df.limit(10).toPandas().to_csv('work/task3-whole.csv',header = False, index = False, sep='\t')